# Lua examples

We are using Redis Server 6

This allows us to use RESP6 in our lua scripts.
The python module hasn't started to implement RESP3 but that is fine for us.

Coming from python table access like table['key'] = value comes very handy

For the start the lua scripts will be more in the context of building views from our core redis storage. So they can be slightly complex.

In [ ]:
import redis

from fastcounting import helper, views, store, files, system

r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

# Unique list of all accounts in use
We achieve this by iterating over all atomic transactions in the 'account:atomic' sorted set.
The RESP3 lua data we get from ZRANGEBYSCORE looks like this:

{{"171255"; {["double"]=135}}; {"75069"; {["double"]=135}}; {"169292"; {["double"]=650}};

In [ ]:
lua_accounts = """
redis.setresp(3)
local accounts = {}
local hash = {}
for i, value in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
    local account = value[2]['double']
    if not hash[account] then
        accounts[#accounts+1] = account
        hash[account] = true
    end
end
return accounts
"""
response = r.eval(lua_accounts, 1, 'account:atomic', 0, 9999999, 'WITHSCORES')

# Sum up all accounts and return a 1 dimensional list to python RESP2 client

If the python client would support resp3 we would be finished after the first for loop.

In our case we have two more for loops. First we create 2 lists, than we concat those 2 lists. That should be fine for now.

In [ ]:
lua_sum = """
redis.setresp(3)
local result = {}
local hash = {}
for i, value in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
    local account = value[2]['double']
    local atomic = value[1]
    if result[account] == nil then result[account] = 0 end
    result[account] = result[account] + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end

local xaccounts = {}
local xsums = {}
for xaccount, xsum in pairs(result) do
    xaccounts[#xaccounts+1] = xaccount
    xsums[#xsums+1] = xsum
end

for i=1, #xsums do
    xaccounts[#xaccounts + 1] = xsums[i]
end
return xaccounts
"""

In [ ]:
response = r.eval(lua_sum, 1, 'account:atomic', 0, 9999999, 'WITHSCORES')

In [ ]:
response